In [1]:
using HDF5, JLD, DataFrames, Distributions, MLBase, StatsBase, PlotlyJS, MXNet
include("MyFunctions.jl");

In [2]:
fullDataSTD = load("dataV2.jld")["dataFilteredSTD"];
bigY = load("dataV2.jld")["bigY"];
Ysum = Array{Float64}(nrow(bigY),1);
Ysum = convert(Array{Float64}, bigY[:P_CW_P_ACW]);

In [3]:
subSampledData = Array(Float64,100000,size(fullDataSTD)[2])
subSampledLabels = Array(Float64,100000);

In [4]:
#Using Entire Dataset
samplesize  = 1000
trainCap = convert(Int32, (round((size(fullDataSTD)[1]*0.95)) - round((size(fullDataSTD)[1]*0.95))%samplesize))
if (size(subSampledData)[1] - trainCap)%samplesize == 0
    testBase = trainCap + 1
else
    testBase = size(fullDataSTD)[1] - ((size(fullDataSTD)[1] - trainCap) + (samplesize - (size(fullDataSTD)[1] - trainCap)%samplesize)) + 1
end
println(trainCap)
println(testBase)
TrainInput = fullDataSTD[1:trainCap,:]'
TrainOutput = Ysum[1:trainCap]
ValidationInput = fullDataSTD[testBase:end,:]'
ValidationOutput = Ysum[testBase:end];

650000
650001


In [5]:
TrainInput = fullDataSTD[1:trainCap,:]'
TrainOutput = Ysum[1:trainCap]
ValidationInput = fullDataSTD[testBase:end,:]'
ValidationOutput = Ysum[testBase:end];

In [6]:
# how to set up data providers using data in memory
batchsize = 1000 # can adjust this later, but must be defined now for next line
trainprovider = mx.ArrayDataProvider(:data => TrainInput, batch_size=batchsize, shuffle=true, :label => TrainOutput)
evalprovider = mx.ArrayDataProvider(:data => ValidationInput, batch_size=batchsize, shuffle=true, :label => ValidationOutput)

MXNet.mx.ArrayDataProvider(Array{Float32,N}[
Float32[1.25364 -0.350692 … -0.871586 -0.423033; 1.19876 -0.207438 … -0.683604 0.154441; … ; 0.128056 -0.779462 … 0.522699 -0.31544; 0.719975 -1.0043 … -0.930279 -0.490822]],Symbol[:data],Array{Float32,N}[
Float32[0.0 0.027 … 0.885 0.0]],Symbol[:label],1000,34225,true,0.0f0,0.0f0,MXNet.mx.NDArray[mx.NDArray{Float32}(181,1000)],MXNet.mx.NDArray[mx.NDArray{Float32}(1000,)])

In [24]:
data = mx.Variable(:data)
label = mx.Variable(:label)
net = @mx.chain     mx.Variable(:data) =>
                    mx.Dropout(p=0.1) =>
                    mx.FullyConnected(num_hidden=150) =>
                    mx.Activation(act_type=:tanh) =>
                    mx.Dropout(p=0.2) =>
                    mx.FullyConnected(num_hidden=100) =>
                    mx.Activation(act_type=:tanh) =>
                    mx.FullyConnected(num_hidden=1) =>        
                    #mx.LinearRegressionOutput(label)
                    mx.LogisticRegressionOutput(label)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x00007ffda760cd80))

In [25]:
model = mx.FeedForward(net, context=mx.cpu())
# set up the optimizer: select one, explore parameters, if desired
#optimizer = mx.SGD(lr=0.01, momentum=0.9, weight_decay=0.00001, grad_clip=10)
optimizer = mx.RMSProp()

MXNet.mx.RMSProp(MXNet.mx.RMSPropOptions(0.001,0.9,1.0e-6,0,1.0e-5,MXNet.mx.LearningRate.Fixed(0.001)),#undef)

In [26]:
# train, reporting loss for training and evaluation sets
# initial training with small batch size, to get to a good neighborhood
batchsize = 200
mx.fit(model, optimizer, initializer=mx.XavierInitializer(distribution = mx.xv_uniform, regularization = mx.xv_avg, magnitude = 1),
eval_metric=mx.MSE(), trainprovider, eval_data=evalprovider, n_epoch = 20)
#mx.fit(model, optimizer, trainprovider, n_epoch=20, eval_data=evalprovider, eval_metric=mx.Accuracy())
# more training with the full sample
batchsize = samplesize
mx.fit(model, optimizer, eval_metric=mx.MSE(), trainprovider, eval_data=evalprovider, n_epoch = 500)

INFO: Start training on MXNet.mx.Context[CPU0]
INFO: Initializing parameters...
INFO: Creating KVStore...
INFO: TempSpace: Total 3 MB allocated on CPU0
INFO: Start training...
INFO: == Epoch 001/020 ==========
INFO: ## Training summary
INFO:                MSE = 0.0251
INFO:               time = 13.9223 seconds
INFO: ## Validation summary
INFO:                MSE = 0.0214
INFO: == Epoch 002/020 ==========
INFO: ## Training summary
INFO:                MSE = 0.0209
INFO:               time = 7.1056 seconds
INFO: ## Validation summary
INFO:                MSE = 0.0204
INFO: == Epoch 003/020 ==========
INFO: ## Training summary
INFO:                MSE = 0.0200
INFO:               time = 6.6693 seconds
INFO: ## Validation summary
INFO:                MSE = 0.0196
INFO: == Epoch 004/020 ==========
INFO: ## Training summary
INFO:                MSE = 0.0196
INFO:               time = 6.6512 seconds
INFO: ## Validation summary
INFO:                MSE = 0.0196
INFO: == Epoch 005/020 ========

In [27]:
# obtain predictions
plotprovider = mx.ArrayDataProvider(:data => ValidationInput, :label => ValidationOutput)
fit = mx.predict(model, plotprovider)
println("RMSE: $(root_mean_squared_error(ValidationOutput', fit))")

INFO: TempSpace: Total 47 MB allocated on CPU0


RMSE: 0.12753409439109198


In [28]:
e = rand(1:size(ValidationOutput)[1],10000)
GZsp = ValidationOutput[e]
NNsp = fit[e]
texto = map(x->"Error = "*string(x)[1:7],(abs(GZsp .- NNsp)))
rmsePlot = scatter(x=GZsp, y=NNsp, name="Predicted Values", mode="markers", text=texto,
marker=attr(cmin=0, cmax=1, color=abs(GZsp .- NNsp), colorscale="Bluered"))
layout = Layout(title="Humans Spirality vs NN Spirality", yaxis=attr(title="NNsp"), xaxis=attr(title="GZsp"))
plot([rmsePlot], layout)

In [29]:
realCumulative = zeros(10)
count = zeros(10)
for i in 1:length(fit)
    if ValidationOutput[i] <= .1
        realCumulative[1]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[1]+=1
    elseif ValidationOutput[i] > .1 && ValidationOutput[i] <= .2
        realCumulative[2]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[2]+=1
    elseif ValidationOutput[i] > .2 &&  ValidationOutput[i] <= .3
        realCumulative[3]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[3]+=1
    elseif ValidationOutput[i] > .3 &&  ValidationOutput[i] <= .4
        realCumulative[4]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[4]+=1
    elseif ValidationOutput[i] > .4 &&  ValidationOutput[i] <= .5
        realCumulative[5]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[5]+=1
    elseif ValidationOutput[i] > .5 &&  ValidationOutput[i] <= .6
        realCumulative[6]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[6]+=1
    elseif ValidationOutput[i] > .6 &&  ValidationOutput[i] <= .7
        realCumulative[7]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[7]+=1
    elseif ValidationOutput[i] > .7 &&  ValidationOutput[i] <= .8
        realCumulative[8]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[8]+=1
    elseif ValidationOutput[i] > .8 &&  ValidationOutput[i] <= .9
        realCumulative[9]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[9]+=1
    elseif ValidationOutput[i] > .9
        realCumulative[10]+=root_mean_squared_error(ValidationOutput[i], fit[i])
        count[10]+=1
    end
end
realCumulative
averageRMSES = realCumulative./count;

In [30]:
count

10-element Array{Float64,1}:
 22387.0
  4742.0
  1957.0
  1154.0
   886.0
   632.0
   587.0
   498.0
   601.0
   781.0

In [31]:
rmsePlot = scatter(x=collect(0.05:0.1:1.05), y=averageRMSES, name="Predicted Values", mode="lines+markers")
layout = Layout(title="Average RMSE in different ranges", yaxis=attr(title="Average RMSE"), xaxis=attr(title="Spirality Range"))
plot([rmsePlot], layout)

In [3]:
realCumulative = zeros(10)
countObj = zeros(10)
for i in 1:length(Ysum)
    if Ysum[i] <= .1
        countObj[1]+=1
    elseif Ysum[i] > .1 && Ysum[i] <= .2
        countObj[2]+=1
    elseif Ysum[i] > .2 &&  Ysum[i] <= .3
        countObj[3]+=1
    elseif Ysum[i] > .3 &&  Ysum[i] <= .4
        countObj[4]+=1
    elseif Ysum[i] > .4 &&  Ysum[i] <= .5
        countObj[5]+=1
    elseif Ysum[i] > .5 &&  Ysum[i] <= .6
        countObj[6]+=1
    elseif Ysum[i] > .6 &&  Ysum[i] <= .7
        countObj[7]+=1
    elseif Ysum[i] > .7 &&  Ysum[i] <= .8
        countObj[8]+=1
    elseif Ysum[i] > .8 &&  Ysum[i] <= .9
        countObj[9]+=1
    elseif Ysum[i] > .9
        countObj[10]+=1
    end
end
countObj

10-element Array{Float64,1}:
 448915.0
  92951.0
  39953.0
  23113.0
  17178.0
  12615.0
  11514.0
  10130.0
  11749.0
  16107.0

In [35]:
subSampledData = Array(Float64,100000,size(fullDataSTD)[2])
subSampledLabels = Array(Float64,100000)
max_allowed = 10000
countClasses = zeros(10)
newIndex = 1
for i in 1:length(Ysum)
    if Ysum[i] <= .1 && countClasses[1] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[1]+=1
    elseif Ysum[i] > .1 && Ysum[i] <= .2 && countClasses[2] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[2]+=1
    elseif Ysum[i] > .2 && Ysum[i] <= .3 && countClasses[3] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[3]+=1
    elseif Ysum[i] > .3 && Ysum[i] <= .4 && countClasses[4] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[4]+=1
    elseif Ysum[i] > .4 && Ysum[i] <= .5 && countClasses[5] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[5]+=1
    elseif Ysum[i] > .5 && Ysum[i] <= .6 && countClasses[6] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[6]+=1
    elseif Ysum[i] > .6 && Ysum[i] <= .7 && countClasses[7] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[7]+=1
    elseif Ysum[i] > .7 && Ysum[i] <= .8 && countClasses[8] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[8]+=1
    elseif Ysum[i] > .8 && Ysum[i] <= .9 && countClasses[9] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[9]+=1
    elseif Ysum[i] > .9 && countClasses[10] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = Ysum[i]
        newIndex+=1
        countClasses[10]+=1
    end
end
p = randperm(size(subSampledData)[1]);
subSampledData = subSampledData[p,:];
subSampledLabels = subSampledLabels[p,:];

In [81]:
subSampledData = Array(Float64,74000,size(fullDataSTD)[2])
subSampledLabels = Array(Float64,74000)
max_allowed = 35000
countClasses = zeros(2)
newIndex = 1
for i in 1:length(Ysum)
    if Ysum[i] <= .3 && countClasses[1] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = 0
        newIndex+=1
        countClasses[1]+=1
    elseif Ysum[i] >= .7 && countClasses[2] < max_allowed
        subSampledData[newIndex,:] = fullDataSTD[i,:]'
        subSampledLabels[newIndex] = 1
        newIndex+=1
        countClasses[2]+=1
    end
end
p = randperm(size(subSampledData)[1]);
subSampledData = subSampledData[p,:];
subSampledLabels = subSampledLabels[p,:];

In [36]:
samplesize  = 1000
trainCap = convert(Int32, ((size(subSampledData)[1]*0.9) - (size(subSampledData)[1]*0.9)%samplesize))
if (size(subSampledData)[1] - trainCap)%samplesize == 0
    testBase = trainCap + 1
else
    testBase = size(subSampledData)[1] - ((size(subSampledData)[1] - trainCap) + (samplesize - (size(subSampledData)[1] - trainCap)%samplesize)) + 1
end
TrainInput = subSampledData[1:trainCap,:]'
TrainOutput = subSampledLabels[1:trainCap]
ValidationInput = subSampledData[testBase:end,:]'
ValidationOutput = subSampledLabels[testBase:end];